In [2]:
# import h5 reader
import h5py
import numpy as np
import pandas as pd
import time
from os import listdir
from os.path import isfile, join
import pickle
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib
import scipy.optimize as opt
from matplotlib.ticker import LinearLocator
from scipy.interpolate import griddata, interp1d
import emcee
from joblib import Parallel, delayed, effective_n_jobs
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

work_path = "/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/2d/InverseUniExp/"
h5_path = work_path + "output/fault/"
print("Number of workers available: ", effective_n_jobs(-1))

Number of workers available:  36


# Gaussian predictor

In [3]:
def string(val):
    return "{:.16e}".format(val)

In [4]:
# Import Gaussian-regression related functions
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

# Pre-process the data
from sklearn import preprocessing

# function train_GP
class GP_predictor:
    # Constructor
    def __init__(self, 
                 input_set, 
                 observation_set, 
                 GPkernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-3, 1e3)), 
                 n_optimizers = 9):
        # Scale input data
        self.input_set = [list(x) for x in input_set]
        self.observation_set = [list(x) for x in observation_set]
        self.input_dimension = len(self.input_set[0])
        self.observation_dimension = len(self.observation_set[0])
        self.trainset_length = len(self.input_set)
        
        self.input_scaler = preprocessing.StandardScaler()
        self.input_scaler.fit(np.array(self.input_set))
        
        # Scale output data
        self.observation_scaler = preprocessing.StandardScaler()
        self.observation_scaler.fit(np.array(observation_set))
        
        # Fit Gaussian process
        self.GP = GaussianProcessRegressor(kernel = GPkernel, n_restarts_optimizer = n_optimizers)
        # self.GP = MyGPR(kernel = GPkernel, n_restarts_optimizer = n_optimizers, max_iter = max_iterations)
        self.GP.fit(self.input_scaler.transform(np.array(self.input_set)), 
                    self.observation_scaler.transform(np.array(self.observation_set)))
        
    # Predict on a new input set
    def predict(self, new_input_set):
        # Predict new observation
        new_observation = self.observation_scaler.inverse_transform(
            self.GP.predict(
                self.input_scaler.transform(np.array(list(new_input_set)).reshape([-1, self.input_dimension]))
            )
        )
        
        return new_observation
        
    

In [5]:
# Class RunABatch
class RunABatch:
    # Constructor
    def __init__(self, input_set, work_path, FourierTerms = 16, distanceAbove = 2e-3, 
                 nOfQueryPts = 10, obsFlag = 'fault', fourierFlag = False):
        # Initialize data paths and batch parameters
        self.work_path = work_path
        self.input_set = [tuple(i) for i in input_set]
        self.h5_path = work_path + "output/faultFiles/"
        self.frontsurf_path = work_path + "output/frontsurfFiles/"
        self.FourierTerms = FourierTerms
        self.distanceAbove = distanceAbove
        self.nOfQueryPts = nOfQueryPts
        self.fourierFlag = fourierFlag
        
        # Store all cases in the h5_path
        self.existingCasesFile = self.h5_path + "CaseList.csv"
        
        # Flag for whether cases have been run
        self.casesExcuted = False
        
        # Get existing cases
        self.getExistingCasesOfInputSet()
        
        # Get input_set_toRun
        self.input_set_toRun = list(set(self.input_set) - self.existingCases)
        
        # Run cases
        self.runCases(self.input_set_toRun)
        
        # Get obsevations
        # self.Observations = self.getObservations(self.input_set)
        if obsFlag == 'fault':
            self.Observations = self.getObservations(self.input_set, self.nOfQueryPts)
        elif obsFlag == 'front':
            self.Observations = self.getObservationsFrontSurf(self.input_set, self.nOfQueryPts, self.distanceAbove)
        else:
            self.Observations = self.getObservationsEveryWhere(self.input_set)
            
    # Inline function gets [A, B] list
    def getABDrsThetaFstar(self, fileName):
        A_idx = fileName.find('A')
        B_idx = fileName.find('B')
        Drs_idx = fileName.find('D')
        Theta_idx = fileName.find('t')
        Fstar_idx = fileName.find('f')
        slash_idx = fileName.find('-fault')

        # Change this part !! Before applying to new name convention
        # print("fileName: ", fileName)
        A = float(fileName[A_idx + 1 : B_idx - 1])
        B = float(fileName[B_idx + 1 : Drs_idx - 1])
        Drs = float(fileName[Drs_idx + 3 : Theta_idx - 1])
        Theta = float(fileName[Theta_idx + 5 : Fstar_idx - 1])
        Fstar = float(fileName[Fstar_idx + 5 : slash_idx])
        return (A, B, Drs, Theta, Fstar)
    
    
    # Function get_existing_files_set
    def getExistingCasesOfInputSet(self):
        # Get all .h5 file names as a list
        myPath = self.h5_path
        onlyFiles = [f for f in listdir(myPath) if (isfile(join(myPath, f)) and f[-8 : ] == 'fault.h5')]
        self.existingCases = set([self.getABDrsThetaFstar(f) for f in onlyFiles])
        
    # Function runCases
    def runCases(self, input_set, nThreads=24):
        # Start the timer
        st_time = time.time()
        
        # Edit the directories
        shell_pathR = self.work_path + "RunJobsJP.sh"
        shellRead = open(shell_pathR, 'r')
        list_of_lines = shellRead.readlines()
        shellRead.close()

        # Divide the input_set into nThreads subsets
        if nThreads > len(input_set):
            nRealThreads = len(input_set)
        else:
            nRealThreads = nThreads
        
        if nRealThreads > 0:
            # Split with Real thread numbers
            split_input_set = np.array_split(input_set, nRealThreads)
            split_threadNo = list(range(1, nRealThreads + 1))
            
#             # Print splits
#             for inputSet, tNo in zip(split_input_set, split_threadNo):
#                 print(inputSet, tNo)
            
            # Submit the jobs
            time_thread = Parallel(n_jobs=nRealThreads, backend='threading')(
                delayed(self.runCasesOneThread)(list_of_lines, inputSet, tNo) for inputSet, tNo in zip(split_input_set, split_threadNo)
            )
        else:
            time_thread = [0.]
        
        # Get running time
        time_cost = time.time() - st_time
        
        # Print running time
        # print("Wall clock time to run all inputs: ", time_cost)
        # print("Sum of all thread times: ", sum(time_thread))
        
        # Return from shell
        self.casesExcuted = True
        
        return
    
    # Function runCasesOneThread
    def runCasesOneThread(self, list_of_lines, input_set, threadNo):
        # Start timer
        st_time = time.time()
        
        # Deep copy the list
        write_lines = list_of_lines.copy()
        
        # Set output path
        shell_pathW = self.work_path + "RunJobsJP-" + str(threadNo) + ".sh"
        
        # Extract parameters
        AA = [string(ele[0]) for ele in input_set]
        BB = [string(ele[1]) for ele in input_set]
        DD = [string(ele[2]) for ele in input_set]
        TT = [string(ele[3]) for ele in input_set]
        FF = [string(ele[4]) for ele in input_set]
        
        # Change the lines
        write_lines[9] = "AA=" + str(tuple(AA)).replace(',', '').replace('\'', '') + "\n"
        write_lines[10] = "BB=" + str(tuple(BB)).replace(',', '').replace('\'', '') + "\n"
        write_lines[11] = "DD=" + str(tuple(DD)).replace(',', '').replace('\'', '') + "\n"
        write_lines[12] = "TT=" + str(tuple(TT)).replace(',', '').replace('\'', '') + "\n"
        write_lines[13] = "FF=" + str(tuple(FF)).replace(',', '').replace('\'', '') + "\n"
        write_lines[14] = "threadNo=" + str(threadNo) + "\n"
        
        # Write to file
        shellWrite = open(shell_pathW, 'w')
        shellWrite.writelines(write_lines)
        shellWrite.close()
        
        #Excute the job
        !source $shell_pathW
        
        # Return time
        return time.time() - st_time
    
    # Function getObservations for the input_set
    def getObservations(self, input_set, nOfQueryPts):
        # Initialize Observations
        Observations = []
        
        # Check if the cases have been excuted
        if self.casesExcuted == False:
            return Observations
        
        # Rotation matrix Q
        alpha = 29 / 180 * np.pi
        Q = np.array([[np.cos(alpha), np.sin(alpha)], [-np.sin(alpha), np.cos(alpha)]])
        
        # VS start
        VSstart = np.array([0.006354, 0.003522])
        
        # Loop through all Inputs
        for input_ele in input_set:
            # Open the file
            h5_file = self.h5_path + "A" + string(input_ele[0]) + "_B" + string(input_ele[1]) \
                      + "_DRS" + string(input_ele[2]) + "_theta" + string(input_ele[3]) + "_fStar" + \
                      string(input_ele[4]) + "-fault.h5"
                     
            f = h5py.File(h5_file, 'r')

            # Get time
            times = np.array(f['time']).reshape([-1])
            times = times - np.min(times)
            nOfTSteps = times.shape[0]
            
            # Get coordinates
            coords = np.array(f['geometry']['vertices']) - VSstart
            Qcoords = coords @ Q.transpose()
            XXs = Qcoords[:, 0]
            
            # Get Slip rates [time, nOfNodes, spaceDim]
            SlipRates = np.array(f['vertex_fields']['slip_rate'])
            SlipRatesMag = np.linalg.norm(SlipRates, ord = 2, axis = 2)
            nOfNodes = SlipRatesMag.shape[1]
            
            xqs = 1e-3 * np.linspace(0., 45., nOfQueryPts)
            
            # Store the slip rates
            slip_rate_func = interp1d(XXs, SlipRatesMag, kind = 'cubic')  # [nOfTSteps, nOfNodes]
            
            # Change slip rate x to [nOfNodes, nOfTSteps]
            slip_rate_x = slip_rate_func(xqs).transpose()
            
#             # DEBUG LINES
#             print('nOfTSteps: ', nOfTSteps)
#             print('nOfNodes: ', nOfNodes)
#             print('SlipRatesMag shape: ', SlipRatesMag.shape)
#             print('slip_rate_x shape: ', slip_rate_x.shape)
            
            # Get the observations
            if self.fourierFlag:
                # Find the Fourier coefficients
                FourierTerms = self.FourierTerms
                T = np.max(times)

                # Compute the Fourier terms
                Ks = np.array(range(FourierTerms))
                coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * times)
                VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in slip_rate_x], 0)

                # Compute the fourier coefficients
                # print('time.shape: ', time.shape)
                observation = np.trapz(VxcoskPiTt, x=times)
                # print("observation shape: ", observation.shape)
                # Append the result from this file
                Observations.append(observation)
            else:
                # Find nFourier terms of values, evenly spaced in [0, T]
                T = np.max(times)
                tts = np.linspace(0., T, self.FourierTerms)
                observation_func = interp1d(times, slip_rate_x)
                observation = observation_func(tts).reshape([-1])
                Observations.append(observation)
                
        Observations = np.array(Observations)
        return Observations
    
    # Function getObservationsFrontSurf for the input_set
    def getObservationsFrontSurf(self, input_set, nOfQueryPts, distanceAbove):
        # Initialize Observations
        Observations = []
        
        # Rotation matrix Q
        alpha = 29 / 180 * np.pi
        Q = np.array([[np.cos(alpha), np.sin(alpha)], [-np.sin(alpha), np.cos(alpha)]])
        
        # VS start
        VSstart = np.array([0.006354, 0.003522])
        
        # Query points
        distance_above = distanceAbove  # Distance above the interface
        
        # Set x_up query points
        x_up = 1e-3 * np.linspace(0., 45., nOfQueryPts)
        QueryPts_up = np.stack([x_up, distance_above * np.ones(x_up.shape)], axis = 1)
        QueryPts_dn = np.stack([x_up, -distance_above * np.ones(x_up.shape)], axis = 1)
        
        # nOfNodes
        nOfNodes = len(x_up)
            
        # Check if the cases have been excuted
        if self.casesExcuted == False:
            return Observations
        
        # Loop through all Inputs
        for input_ele in input_set:
            # Open the file
            h5_file = self.frontsurf_path + "A" + string(input_ele[0]) + "_B" + string(input_ele[1]) \
                      + "_DRS" + string(input_ele[2]) + "_theta" + string(input_ele[3]) + "_fStar" + \
                      string(input_ele[4]) + "-domain.h5"
                     
            f = h5py.File(h5_file, 'r')
            
            # Get time
            times = np.array(f['time']).reshape([-1])
            times = times - np.min(times)
            nOfTSteps = times.shape[0]

            # Get coordinates
            coords = np.array(f['geometry']['vertices']) - VSstart
            Qcoords = coords @ Q.transpose()
            
            # Store the slip rates
            slip_rate_x = np.zeros([nOfTSteps, nOfNodes])
            
            # Get Slip rates
            velocity = np.array(f['vertex_fields']['velocity'])
            Qvelocity = velocity @ Q.transpose()
            
            for i in range(nOfTSteps):
                slip_rate_x[i, :] = - griddata(Qcoords, velocity[i, :, 0], QueryPts_up, method = 'cubic') \
                              + griddata(Qcoords, velocity[i, :, 0], QueryPts_dn, method = 'cubic')
            slip_rate_x = slip_rate_x.transpose()
            
            # Get the observations
            if self.fourierFlag:
                # Find the Fourier coefficients
                FourierTerms = self.FourierTerms
                T = np.max(times)

                # Compute the Fourier terms
                Ks = np.array(range(FourierTerms))
                coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * times)
                VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in slip_rate_x], 0)

                # Compute the fourier coefficients
                # print('time.shape: ', time.shape)
                observation = np.trapz(VxcoskPiTt, x=times)
                # print("observation shape: ", observation.shape)
                # Append the result from this file
                Observations.append(observation)
            
            else:
                # Find nFourier terms of values, evenly spaced in [0, T]
                T = np.max(times)
                tts = np.linspace(0., T, self.FourierTerms)
                observation_func = interp1d(times, slip_rate_x)
                observation = observation_func(tts).reshape([-1])
                Observations.append(observation)
        
        Observations = np.array(Observations)
        return Observations
    
    # Function getObservationsFrontSurf for the input_set
    def getObservationsEveryWhere(self, input_set):
        # Initialize Observations
        Observations = []
        
        # Rotation matrix Q
        alpha = 29 / 180 * np.pi
        Q = np.array([[np.cos(alpha), np.sin(alpha)], [-np.sin(alpha), np.cos(alpha)]])
        
        # VS start
        VSstart = np.array([0.006354, 0.003522])
            
        # Check if the cases have been excuted
        if self.casesExcuted == False:
            return Observations
        
        # Loop through all Inputs
        for input_ele in input_set:
            # Open the file
            h5_file = self.frontsurf_path + "A" + string(input_ele[0]) + "_B" + string(input_ele[1]) \
                      + "_DRS" + string(input_ele[2]) + "_theta" + string(input_ele[3]) + "_fStar" + \
                      string(input_ele[4]) + "-domain.h5"
                     
            f = h5py.File(h5_file, 'r')
            
            # Get time
            times = np.array(f['time']).reshape([-1])
            times = times - np.min(times)
            nOfTSteps = len(times)
            
            # Get coordinates
            coords = np.array(f['geometry']['vertices']) - VSstart
            Qcoords = coords @ Q.transpose()
            
            # Get the node list in the observable zone
            nodeList = np.all(
                np.stack(
                    [Qcoords[:, 0] >= 0., Qcoords[:, 0] <= 46.e-3, Qcoords[:, 1] >= -15.e-3, Qcoords[:, 1] <= 15.e-3]
                ), 
                axis = 0 
            )
            
            # Get Slip rates
            velocity = np.array(f['vertex_fields']['velocity'])[:, nodeList, :]
            Qvelocity = velocity @ Q.transpose()
            # print("Qvelocity shape: ", Qvelocity.shape)
            # Collapse all measurements
            slip_rate_x = Qvelocity.reshape([Qvelocity.shape[0], -1]).transpose()
            
            
            
            # Find nFourier terms of values, evenly spaced in [0, T]
            T = np.max(times)
            tts = np.linspace(0., T, len(times) // 5)
            observation_func = interp1d(times, slip_rate_x)
            observation = observation_func(tts).reshape([-1])
            Observations.append(observation)
        
        Observations = np.array(Observations)
        return Observations
    

# Class BayersianInv

In [8]:
# Split a hyper-rectangle into n_gridPts^K rectangles
def divideRects(low_hi, nGridPts):
    low = low_hi[0]
    hi = low_hi[1]
    newLows = np.linspace(low, hi, nGridPts)[0 : -1].transpose()
    delta = (hi - low) / (nGridPts - 1)
    
    # Change newLows into starting points
    shits = np.meshgrid(*newLows)
    newLows = np.array([x.reshape(-1) for x in shits]).transpose()
    newHighs = newLows + delta
    return newLows, newHighs

low_hi = [np.linspace(1., 2., 3), np.linspace(3., 4., 3)]
nGridPts = 3
newLows, newHighs = divideRects(low_hi, nGridPts)

In [9]:
newLows

array([[1. , 1.5, 2. ],
       [1. , 1.5, 3. ],
       [2. , 1.5, 2. ],
       [2. , 1.5, 3. ],
       [1. , 2.5, 2. ],
       [1. , 2.5, 3. ],
       [2. , 2.5, 2. ],
       [2. , 2.5, 3. ]])

In [10]:
newHighs

array([[2. , 2.5, 3. ],
       [2. , 2.5, 4. ],
       [3. , 2.5, 3. ],
       [3. , 2.5, 4. ],
       [2. , 3.5, 3. ],
       [2. , 3.5, 4. ],
       [3. , 3.5, 3. ],
       [3. , 3.5, 4. ]])

In [42]:
# Define Bayersian Inv that solves a Bayersian inversion problem
class BayersianInv:
    # Constructor
    def __init__(self, u_low, u_high, u, y, work_path, FourierTerms = 16, 
                 atol = 1.0e-6, si_eta = 0.5, n_samples = 20, MCMCsteps = 1000, save_path = "./", 
                 distanceAbove = 2e-3, nOfQueryPts = 10, obsFlag = 'fault', fourierFlag = False, 
                 samplesFlag = 'sampling'):
        # Set X has to be compact in R^k
        self.real_u_idx = np.array((u_low != u_high))
        self.u_low = u_low[self.real_u_idx]
        self.u_high = u_high[self.real_u_idx]
        self.dummy_u = u_low[~self.real_u_idx]
        
        
        # For each iteration, needs a region to build the grid points
        self.u_lows = [u_low]
        self.u_highs = [u_high]
        
        # Store all the Rectangles
        self.Rects = [[u_low, u_high]] 
        self.RectsArea = [np.product(u_high - u_low)]
        
        # The true value of u, as well as the observation y to be inverted
        self.u = u
        self.y = y
        
        # Other parameters
        self.input_dim_All = len(u_low)
        self.input_dim = len(self.u_low)
        self.output_dim = len(self.y)
        self.work_path = work_path
        self.FourierTerms = FourierTerms
        self.atol = atol
        self.si_eta = si_eta
        self.n_samples = n_samples
        self.MCMCsteps = MCMCsteps
        self.save_path = save_path
        
        # Keep record of distanceAbove and nOfQueryPts in the field of view
        self.distanceAbove = distanceAbove
        self.nOfQueryPts = nOfQueryPts
        
        # Get where the observations are from
        self.obsFlag = obsFlag
        self.fourierFlag = fourierFlag
        self.samplesFlag = samplesFlag
        
        # Keep records of the sampled points and the corresponding observations after iteration
        self.U = np.empty([0, self.input_dim])
        self.O = np.empty([0, self.output_dim])
        self.iterations = 0
        
        # Keep record of eta, MaxMinLenRatio, GaussianProcess Emulator, average error on the sampled points, 
        # and empirical mean and stdv of the posterior after each iteration
        self.etas = []
        self.maxMinDistRatio = []
        self.GPs = []
        self.avg_errors = []
        self.mean = []
        self.stdv = []
        
        # Maximum likelihood points
        self.maxLikelihoodUs = []
        self.maxLikelihoodUsPropL2Error = []
        self.maxLikelihoodObsPropL2Error = []
    
    # Cast u into the same length of u_low, with dummy u
    def castU(self, u_real):
        # print("u_real shape: ", u_real.shape)
        u_real_this = u_real.reshape([-1, self.real_u_idx.sum()])
        # print("u_real_this shape: ", u_real_this.shape)
        
        res = np.zeros([u_real_this.shape[0], len(self.u)])
        res[:, self.real_u_idx] = u_real_this
        res[:, ~self.real_u_idx] = self.dummy_u
        return res
    
    # Get the accumulative probability function
    def log_prob(self, u, y):
        # Apply hard constraints
        u_reshape = u.reshape([-1, self.input_dim])
        normal_idx = np.all(
            np.concatenate(
                [u_reshape >= self.u_low, u_reshape <= self.u_high], axis = 1
            ), 
            axis = 1
        )
       
        # First prior is uniform distribution
        res = np.ones(len(u_reshape))
        res[~normal_idx] = -np.inf
        
        # Add all posteriors in each iteration (since this is log of probability density)
        if (np.sum(normal_idx) > 0):
            for i in range(self.iterations):
                res[normal_idx] += -0.5 / self.etas[i] ** 2 * (
                    np.sum(
                        (y - self.GPs[i].predict(u_reshape[normal_idx])) ** 2, 
                        axis = 1
                    )
                )
        
        # Return the log_probability at the current iteration
        return res
    
    # Compute statistics: sample and calculate mean and stdv
    def compute_stats(self, n_samples, n_steps = 1_000):
        # Sample for more points to update the empirical statistics
        sampler = emcee.EnsembleSampler(n_samples, 
                                        self.input_dim, 
                                        self.log_prob, args=[y], 
                                        vectorize = True)

        # Initialize uniformly as the starting point
        p0 = np.random.uniform(size = [n_samples, self.input_dim]) * (self.u_high - self.u_low) + self.u_low

        # Get the result
        sampler.run_mcmc(p0, n_steps)
        samples = sampler.get_last_sample().coords
        self.mean.append(np.mean(samples, axis = 0))
        self.stdv.append(np.std(samples, axis = 0))
        
        # Get maximum likelihood estimate
        fun = lambda u: -self.log_prob(u, self.y)
        newCenter = opt.minimize(fun, x0 = self.mean[-1], 
                                 bounds = [(self.u_low[i], self.u_high[i]) for i in range(len(self.u_low))]
                                ).x
        
        self.maxLikelihoodUs.append(newCenter)
        
        self.maxLikelihoodUsPropL2Error.append(
            np.linalg.norm(self.u - self.maxLikelihoodUs[-1]) 
            / np.linalg.norm(self.u)
        )
        self.maxLikelihoodObsPropL2Error.append(
            np.linalg.norm(self.y - self.GPs[-1].predict(self.maxLikelihoodUs[-1])) 
            / np.linalg.norm(self.y)
        )
        
        # Find the new rectangle
        
        
#         # Function to determine if a point is in the rectangles
#         def isInRectangle(setOfRects, pt):
#             res = [ for rec in setOfRects]
        
    # Get average error of a batch
    def get_avg_error_of_a_batch(self, myBatch):
        return np.mean(np.sum((myBatch.Observations - self.y) ** 2, axis = 1))
    
    # Run one iteration
    def runOneIteration(self, n_samples, n_stat_samples):
        # Switch based on samplesFlag
        if self.samplesFlag == 'sampling':
            # Sample from the current distribution for u and get the current observations
            sampler = emcee.EnsembleSampler(n_samples, 
                                            self.input_dim, 
                                            self.log_prob, args=[y], 
                                            vectorize = True)

            # Initialize uniformly as the starting point
            p0 = np.random.uniform(size = [n_samples, self.input_dim]) * (self.u_high - self.u_low) + self.u_low

            # Get the result
            sampler.run_mcmc(p0, self.MCMCsteps)
            self.samples = sampler.get_last_sample().coords

            # If there is self_etas, this is to make sure MaxMinDistRatio does not go too large
            if len(self.etas) > 0:
                eta_times = 0
                while (eta_times < 2) and (MaxMinDistRatio(self.samples) > 100.):
                    # Double the last eta
                    self.etas[-1] = 2. * self.etas[-1]
                    eta_times += 1

                    # Re-sample
                    sampler = emcee.EnsembleSampler(n_samples, 
                                                    self.input_dim, 
                                                    self.log_prob, args=[y], 
                                                    vectorize = True)

                    # Initialize uniformly as the starting point
                    p0 = np.random.uniform(size = [n_samples, self.input_dim]) * (self.u_high - self.u_low) + self.u_low

                    # Get the result
                    sampler.run_mcmc(p0, self.MCMCsteps)
                    self.samples = sampler.get_last_sample().coords
                    MCsteps = 0
                    while (MCsteps < self.MCMCsteps / 10) and (MaxMinDistRatio(self.samples) > 10.):
                        sampler.run_mcmc(sampler.get_last_sample(), 1)
                        self.samples = sampler.get_last_sample().coords
                        MCsteps += 1

                # print("eta_times, MaxMinDistRatio: ", eta_times, MaxMinDistRatio(self.samples))
        
        elif self.samplesFlag == 'grid':
            # Construct the samples
            gridPts = np.linspace(self.u_lows[-1], self.u_highs[-1], n_samples).transpose()
            mesh = np.meshgrid(*gridPts)
            
            # Generate the mesh
            mesh = [mesh[i].reshape(-1) for i in range(len(mesh))]
            self.samples = np.stack(mesh, axis = 1)

        # Run the forward map with the samples
        myBatch = RunABatch(self.castU(self.samples), self.work_path, FourierTerms = self.FourierTerms, 
                            distanceAbove = self.distanceAbove, nOfQueryPts = self.nOfQueryPts, 
                            obsFlag = self.obsFlag, fourierFlag = self.fourierFlag)
        self.U = np.append(self.U, self.samples, axis = 0)
        self.O = np.append(self.O, myBatch.Observations, axis = 0)

        # Get a new Gaussian process
        # myGP = GP_predictor(self.samples, myBatch.Observations)
        myGP = GP_predictor(self.U, self.O)

        # Update the recorded variables of the process
        self.GPs.append(myGP)
        self.iterations += 1
        self.etas.append(self.si_eta)
        self.avg_errors.append(self.get_avg_error_of_a_batch(myBatch))
        self.maxMinDistRatio.append(MaxMinDistRatio(self.samples))
        
        # Update stats
        self.compute_stats(n_stat_samples)
        
        # Converging flag False
        return False
    
    # Run function
    def run(self, n_iter_max = 10, n_stat_samples = 1000):
        for i in range(n_iter_max):
            # In each iteration
            print("="*30, " Iteration ", str(self.iterations), " ", "="*30)
            
            # Run one iteration
            start_time = time.time()
            converged = self.runOneIteration(self.n_samples, n_stat_samples)
            end_time = time.time()
            print("Time cost: ", str(end_time - start_time), " s")
            # print("Average error in this iteration: ", self.avg_errors[-1])
            print("Maximum to minimum distance in the sample points: ", self.maxMinDistRatio[-1])
            # print("Mean of the posterior after this iteration: ", self.mean[-1])
            # print("Stdv of the posterior after this iteration: ", self.stdv[-1])
            print("Max likelihood estimate of the posterior after this iteration: ", self.maxLikelihoodUs[-1])
            print("L2 error of u: ", self.maxLikelihoodUsPropL2Error[-1])
            print("L2 error of y: ", self.maxLikelihoodObsPropL2Error[-1])
        # Save the current model to files
        self.save()
        
    # Save the model to files
    def save(self):
        savePath = self.save_path + "models/A{0}_B{1}_DRS{2}_theta{3}_fStar{4}_nOfQPts{5}_eta{6}-{7}-Fourier-{8}.pickle".format(self.u[0], self.u[1], self.u[2], self.u[3], self.u[4], self.nOfQueryPts, self.si_eta, self.obsFlag, self.fourierFlag)
        with open(savePath, 'wb') as file:
            pickle.dump(self, file)
    
    # Plot the regression process into a gif file
    def plotGIF(self):
        plotSeries(self, self.u, self.y, self.save_path)

        
# ===================================== Other Auxiliary functions ==========================================
# Function MaxMinDistRatio, compute the ratio of maximum to minimum distance among pts, 
# for the gaussian process to converge with a reasonable length-scaled kernel, 
# this value should not be too large
def MaxMinDistRatio(pts):
    matrix = [[np.linalg.norm(pts[i] - pts[j]) for j in range(i + 1, len(pts))] for i in range(len(pts))]
    vec = [x for y in matrix for x in y]
    return max(vec) / min(vec)


# Calculate the corresponding (to y) log likelihood of iter_step on u
def log_prob_best(self, iter_step, u, y):
    # Apply hard constraints
    u_reshape = u.reshape([-1, self.input_dim])
    normal_idx = np.all(np.concatenate([u_reshape >= self.u_low, u_reshape <= self.u_high], axis = 1), axis = 1)

    # First prior is uniform distribution
    res = np.ones(len(u_reshape))
    res[~normal_idx] = -np.inf

    # Add all posteriors in each iteration
    if (np.sum(normal_idx) > 0):
        for i in range(iter_step + 1):
            res[normal_idx] += -0.5 / self.etas[i] ** 2 * (
                np.sum(
                    (y - self.GPs[i].predict(u_reshape[normal_idx])) ** 2, 
                    axis = 1
                )
            )

    # Return the log_probability at the current iteration
    return res

# Plot the given regression process on (analytical_u, analytical_y) and save the GIF
def plotSeries(self, analytical_u, analytical_y, save_path, dpi_value = 300):
    # Calculate U_plot
    minU = self.u_low
    maxU = self.u_high
    nOfGridPoints = 100

    xis = []
    for i in range(minU.shape[0]):
        xis.append(np.linspace(minU[i], maxU[i], nOfGridPoints))

    # Generate grid and draw predictions
    UPlotGrid = np.meshgrid(xis[0], xis[1])
    UPlotGrid = np.stack(UPlotGrid, axis = 2)
    UPlotGridFat = UPlotGrid.reshape([nOfGridPoints * nOfGridPoints, maxU.shape[0]])
    
    gifName = save_path + "figures/A{0}_B{1}.gif".format(analytical_u[0], analytical_u[1])
    
    writer = imageio.get_writer(gifName, mode='I', duration = 1.0)
    
    # Maximum likelihood points
    self.maxLikelihoodUs = []
    self.maxLikelihoodUsPropL2Error = []
    self.maxLikelihoodObsPropL2Error = []
    
    # Plot the series
    for i in range(0, self.iterations):
        plt.figure(figsize = (7, 6), dpi = dpi_value)
        YPlotGridFat = log_prob_best(self, i, UPlotGridFat, analytical_y)
        YPlotGridFat = YPlotGridFat - np.max(YPlotGridFat)
        
        # Get optimized u
        idx = np.argmax(YPlotGridFat)
        
        # self.maxLikelihoodUs.append(UPlotGridFat[idx])
        
        fun = lambda u: -log_prob_best(self, i, u, analytical_y)
        newCenter = opt.minimize(fun, x0 = UPlotGridFat[idx], 
                                 bounds = [(self.u_low[i], self.u_high[i]) for i in range(len(self.u_low))]
                                ).x
        
        # DEBUG LINES
        # print("=" * 60)
        # print("i, u = ", i, newCenter)
        
        # print("-fun(u_opt), log_prob_best(u_opt): ", -fun(newCenter), log_prob_best(self, i, newCenter, analytical_y))
        # print("-fun(grid_opt), log_prob_best(grid_opt): ", -fun(UPlotGridFat[idx]), log_prob_best(self, i, UPlotGridFat[idx], analytical_y))
        self.maxLikelihoodUs.append(newCenter)
        
        self.maxLikelihoodUsPropL2Error.append(
            np.linalg.norm(analytical_u - self.maxLikelihoodUs[-1]) 
            / np.linalg.norm(analytical_u)
        )
        self.maxLikelihoodObsPropL2Error.append(
            np.linalg.norm(analytical_y - self.GPs[i].predict(self.maxLikelihoodUs[-1])) 
            / np.linalg.norm(analytical_y)
        )
        
        YPlotGrid = YPlotGridFat.reshape([nOfGridPoints, nOfGridPoints])
        cp = plt.contourf(UPlotGrid[:, :, 0], UPlotGrid[:, :, 1], np.maximum(YPlotGrid, -5.))
        
        # Give the color bar
        cbar = plt.colorbar(cp)
        plt.clim([-5., 0.])
        cbar.set_label('$-\\Phi(u)$', fontsize = 20)
        
        # Scatter the sample points
        plt.scatter(self.U[ :(i + 1) * self.n_samples, 0], 
                    self.U[ :(i + 1) * self.n_samples, 1], s = 1, color = 'white')
        plt.scatter(analytical_u[0], analytical_u[1], s = 15, color = 'red')
        plt.xlabel('$u_1$', fontsize = 20)
        plt.ylabel('$u_2$', fontsize = 20)
        plt.title("The " + str(i) + " th iteration")
        figName = save_path + "figures/shit" + str(i) + ".png"
        plt.savefig(figName, dpi = dpi_value)
        
        # Save the figures into a gif
        image = imageio.imread(figName)
        writer.append_data(image)
        !rm $figName
    writer.close()
    
    # Save a figure of L2PropError of u
    plt.figure(figsize = (7, 6), dpi = dpi_value)
    plt.scatter(range(1, 1 + len(self.maxLikelihoodUsPropL2Error)), self.maxLikelihoodUsPropL2Error, s = 20)
    plt.xlabel('Iteration', fontsize = 20)
    plt.ylabel('$\|u-u_{true}\|/\|u_{true}\|$', fontsize = 20)
    # plt.ylim([0., 0.01])
    plt.title('Relative L2 error of $u$', fontsize = 20)
    figName = save_path + "figures/A{0}_B{1}_propL2UError.png".format(analytical_u[0], analytical_u[1])
    plt.savefig(figName, dpi = dpi_value)

    # Save a figure of L2PropError of y
    plt.figure(figsize = (7, 6), dpi = dpi_value)
    plt.scatter(range(1, 1 + len(self.maxLikelihoodUsPropL2Error)), self.maxLikelihoodObsPropL2Error, s = 20)
    plt.xlabel('Iteration', fontsize = 20)
    plt.ylabel('$\|y-y_{true}\|/\|y_{true}\|$', fontsize = 20)
    # plt.ylim([0., 0.01])
    plt.title('Relative L2 error of $y$', fontsize = 20)
    figName = save_path + "figures/A{0}_B{1}_propL2YError.png".format(analytical_u[0], analytical_u[1])
    plt.savefig(figName, dpi = dpi_value)
    
    

In [43]:
# Load from file
def loadInvModel(model_path):
    with open(model_path, 'rb') as file:
        myInv = pickle.load(file)
    return myInv

# Test an inversion

In [44]:
# Test run a batch
input_set = [[0.011, 0.016, 3e-6, 0.006, 0.58], [0.012, 0.015, 2e-6, 0.003, 0.6]]
distanceAbove = 1e-3
nOfQueryPts = 10
fTerms = 32
obsFlag = 'everywhere'
fourierFlag = True
myBatch = RunABatch(input_set, work_path, fTerms, distanceAbove, nOfQueryPts, obsFlag, fourierFlag)

In [45]:
myBatch.input_set

[(0.011, 0.016, 3e-06, 0.006, 0.58), (0.012, 0.015, 2e-06, 0.003, 0.6)]

In [46]:
# Generate test batch
u_low = np.array([0.005, 0.01, 3e-6, 0.006, 0.58])
u_high = np.array([0.015, 0.02, 3e-6, 0.006, 0.58])
nSamples = 24
siEta = 0.5
print("Test input set: ", myBatch.input_set)
print("Work path: ", work_path)

Test input set:  [(0.011, 0.016, 3e-06, 0.006, 0.58), (0.012, 0.015, 2e-06, 0.003, 0.6)]
Work path:  /home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/2d/InverseUniExp/


In [47]:
### Run an inversion test:
testCase_idx = 0
u = myBatch.input_set[testCase_idx]
y = myBatch.Observations[testCase_idx]
myInv = BayersianInv(u_low, u_high, u, y, work_path, FourierTerms = fTerms, 
                     n_samples = nSamples, 
                     si_eta = siEta, 
                     nOfQueryPts = nOfQueryPts, 
                     distanceAbove = distanceAbove, 
                     obsFlag = obsFlag, 
                     fourierFlag = fourierFlag
                    )
myInv.run(n_iter_max = 10)

# # Save to file
# import pickle

# #save it
# with open(f'./models/myInvA4_B6.pickle', 'wb') as file:
#     pickle.dump(myInv, file) 

==============================  Iteration  0   ==============================
u_real shape:  (24, 2)
u_real_this shape:  (24, 2)
Running case A1.3925364683702118e-02_B1.9314657921045027e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 1
Running case A1.2003028153109767e-02_B1.9492421808626939e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 4
Running case A8.5027970855033278e-03_B1.2693636396063655e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 3
Running case A1.2786151491779085e-02_B1.0110076564955264e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 9
Running case A8.9534469103554667e-03_B1.3667032308893315e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 6
Running case A9.6771468779768242e-03_B1.2661586425667249e-02_DRS3.0000000000000001e-06_theta6.00

KeyboardInterrupt: 

In [151]:
myInv.run(n_iter_max = 10)

==============================  Iteration  10   ==============================
Running case A0.011926963874247775_B0.019768318534366655_DRS1.6543216238671061e-06_theta0.00682408321397031_fStar0.7674718300083564 on thread 6
Running case A0.013314954546971762_B0.010546276609171014_DRS3.7418684657357646e-06_theta0.007442480322227146_fStar0.6349678367275696 on thread 7
Running case A0.011942297817437768_B0.016807952268964865_DRS4.655484790908336e-06_theta0.003285688962390622_fStar0.6173182493210446 on thread 10
Running case A0.005464544690624372_B0.0114083902526878_DRS3.5451952146916e-06_theta0.007886955965364493_fStar0.6387735222650351 on thread 4
Running case A0.009961878728712598_B0.016171211206571286_DRS3.0929613454909485e-06_theta0.0022266190203435827_fStar0.7076868962322992 on thread 3
Running case A0.013217974389648689_B0.010466968526653115_DRS2.4242969085295134e-06_theta0.00928052877805912_fStar0.7871356052404075 on thread 5
Running case A0.010131895004100219_B0.013377811789038272_

Running case A0.013291388553355774_B0.015954759941280244_DRS1.063491844369818e-06_theta0.008388246647868771_fStar0.6849196910380023 on thread 9
Finished in 72 s!

Running case A0.007259186036287077_B0.014469215888645363_DRS2.7478464030378163e-06_theta0.008475940222672104_fStar0.7464439047603526 on thread 3
Running case A0.008785746241407062_B0.013520010419788923_DRS1.6246441538536337e-06_theta0.006017322118451296_fStar0.7609444650602988 on thread 6
Finished in 73 s!

Finished in 73 s!

Running case A0.007331717737834899_B0.011954217537912957_DRS4.366925976096801e-06_theta0.007139274190934639_fStar0.7253415018765555 on thread 16
Running case A0.009068881486413025_B0.014468628248277719_DRS3.677055834527724e-06_theta0.007473768275249147_fStar0.6316844388320169 on thread 7
Finished in 73 s!

Running case A0.005982001287626119_B0.014956981824867719_DRS2.0462221671805743e-06_theta0.004410516076879209_fStar0.7182743433475032 on thread 10
Finished in 73 s!

Finished in 73 s!

Running case A0.0

Finished in 73 s!

Finished in 72 s!

Finished in 73 s!

Finished in 73 s!

Finished in 73 s!

Running case A0.01421921069465193_B0.016221178393907566_DRS4.5696049264243e-06_theta0.007577242724055171_fStar0.6335598207376406 on thread 9
Running case A0.012485137285983407_B0.013023110206904198_DRS3.7221776208194225e-06_theta0.002213883666710898_fStar0.769459864638345 on thread 11
Running case A0.014771397257169265_B0.01848512691780152_DRS4.231687172571893e-06_theta0.0020634716701337075_fStar0.6855684190693793 on thread 3
Running case A0.006699035596841501_B0.0120276581215995_DRS4.128920476732567e-06_theta0.006077529710671526_fStar0.7353273189145951 on thread 10
Running case A0.008979066936883408_B0.014620669510360819_DRS4.30228868180984e-06_theta0.00752827747435302_fStar0.7555716343931392 on thread 8
Finished in 72 s!

Finished in 72 s!

Finished in 72 s!

Finished in 73 s!

Finished in 72 s!

Running case A0.013011142745434873_B0.013725895558387687_DRS4.590210671550439e-06_theta0.006288

Finished in 73 s!

Running case A0.011770891915159445_B0.014645547971724883_DRS2.5749038601529713e-06_theta0.005549345351402854_fStar0.702660502414964 on thread 10
Finished in 73 s!

Finished in 73 s!

Running case A0.009411718373113248_B0.015544391849758076_DRS4.5690244203814036e-06_theta0.0065832500808729886_fStar0.7606790851154164 on thread 5
Running case A0.013621682482775259_B0.014815487371097563_DRS2.0317556631999925e-06_theta0.001353036028369814_fStar0.6242910051149322 on thread 6
Running case A0.007959841699791819_B0.014512160861829396_DRS4.502192670467871e-06_theta0.008658041917878547_fStar0.7803512582536551 on thread 3
Running case A0.011274517451760823_B0.014263123310955013_DRS4.978169776771841e-06_theta0.004406064661003088_fStar0.6769389769186407 on thread 12
Running case A0.01455023828904022_B0.011926224632583565_DRS2.5563930397782755e-06_theta0.007086509504260204_fStar0.6619744813783182 on thread 9
Finished in 73 s!

Running case A0.012808256254380361_B0.0167711960012967_

Running case A0.008932589035755502_B0.01345028665040536_DRS4.60217908500416e-06_theta0.007861746070532333_fStar0.7056704338948906 on thread 14
Running case A0.01008793812946664_B0.0161505537142426_DRS1.7469191063454052e-06_theta0.00699448540434311_fStar0.6860641068020126 on thread 15
Finished in 73 s!

Running case A0.009850248112338058_B0.015647439524066057_DRS1.908038855985473e-06_theta0.0067337285554716865_fStar0.7413507788645959 on thread 5
Finished in 73 s!

Finished in 73 s!

Finished in 73 s!

Running case A0.006259593813604161_B0.013360451065924562_DRS1.6033264267820885e-06_theta0.0038704080035069947_fStar0.7703803923597534 on thread 12
Finished in 73 s!

Finished in 74 s!

Running case A0.01440458365085251_B0.017679553654506595_DRS2.885440842551105e-06_theta0.0077015781242012376_fStar0.6060759782569417 on thread 16
Finished in 73 s!

Finished in 73 s!

Running case A0.012251587593022412_B0.011628261020289953_DRS3.2341913084723175e-06_theta0.00839376806518559_fStar0.73946732957

...

Finished in 74 s!

Finished in 74 s!

Finished in 74 s!

Finished in 74 s!

Running case A0.009671889379065463_B0.012551330330999763_DRS1.6279993477650776e-06_theta0.0012083743935192735_fStar0.6580748232032793 on thread 8
Running case A0.009318048848976771_B0.013979003993099691_DRS4.65207881162469e-06_theta0.005518228928222634_fStar0.6349645481034338 on thread 15
Running case A0.014804448546382784_B0.014410364322868137_DRS2.381875984469478e-06_theta0.008703212193084441_fStar0.6644195309323473 on thread 5
Finished in 74 s!

Running case A0.013766603815740778_B0.016803781508151046_DRS2.876324951107472e-06_theta0.009715874861946389_fStar0.6300463544912406 on thread 9
Finished in 74 s!

Finished in 74 s!

Running case A0.010485447847785333_B0.015688960220958652_DRS1.6369432055916886e-06_theta0.008770502066745062_fStar0.7316920690028532 on thread 11
Running case A0.006013400639485306_B0.015659863537879154_DRS4.405027464399679e-06_theta0.008679680161991877_fStar0.7221940182380832 on thread 1

Running case A0.010207450616324661_B0.01910050421723935_DRS2.853579183551214e-06_theta0.008245324531696895_fStar0.6726235182462276 on thread 1
Running case A0.007982993288560305_B0.016203477250780984_DRS3.2784086410033705e-06_theta0.007258294552067238_fStar0.7092441364596566 on thread 15
Running case A0.006107629718968771_B0.011835335363488313_DRS2.15733656851736e-06_theta0.004414487719939847_fStar0.6870104504295595 on thread 16
Finished in 72 s!

Running case A0.010615466210947985_B0.011835217983947904_DRS4.750186429552032e-06_theta0.005429253263234637_fStar0.6902292072383088 on thread 6
Finished in 72 s!

Finished in 72 s!

Running case A0.008963283397894352_B0.015691780461109424_DRS3.0046252644258913e-06_theta0.00792186188421927_fStar0.7470579978138191 on thread 8
Finished in 72 s!

Running case A0.009785687153783182_B0.018960014553255695_DRS1.2779141130446873e-06_theta0.00901813414307909_fStar0.7181479724096197 on thread 10
Finished in 72 s!

Running case A0.01142114085211563_B0.01

Finished in 73 s!

Running case A0.0065647719586397965_B0.01659903018527009_DRS1.4441101350993562e-06_theta0.0071910167844526805_fStar0.773365370538758 on thread 8
Finished in 74 s!

Running case A0.005066721493139451_B0.014279842623065088_DRS2.9519068736508473e-06_theta0.005636845147381492_fStar0.7135136368253744 on thread 15
Finished in 73 s!

Finished in 74 s!

Finished in 74 s!

Running case A0.011047161906272257_B0.015035561456428696_DRS2.8832543948624984e-06_theta0.007821897461160436_fStar0.6314766348885879 on thread 1
Running case A0.005302564824026877_B0.011942771369681372_DRS2.862904984375612e-06_theta0.006734311080935024_fStar0.7467844379469517 on thread 6
Finished in 73 s!

Running case A0.01139792848833672_B0.011269744237528903_DRS4.297442611384697e-06_theta0.006720667510275434_fStar0.5884170503569265 on thread 9
Finished in 73 s!

Finished in 73 s!

Running case A0.008313737651687844_B0.014200092734149617_DRS2.3720463515349593e-06_theta0.005286795049942115_fStar0.756221045

Running case A0.00826150169973684_B0.012810726903230657_DRS4.56662737942745e-06_theta0.005788458086803264_fStar0.6799612061009025 on thread 2
Finished in 73 s!

Finished in 73 s!

Running case A0.010625111639616595_B0.017701528758541737_DRS2.2670774523198483e-06_theta0.009657714190341397_fStar0.745278714760249 on thread 15
Running case A0.014162913661887922_B0.01772173872616257_DRS1.1535402019680787e-06_theta0.003617200814445581_fStar0.7630262752713285 on thread 5
Finished in 72 s!

Finished in 72 s!

Running case A0.011026780782338425_B0.015394447315763789_DRS4.539235385653469e-06_theta0.008307742294680995_fStar0.678677274586138 on thread 16
Running case A0.014359324739626823_B0.012174841927588234_DRS3.3530787816563096e-06_theta0.005463999971801251_fStar0.6063875944917604 on thread 13
Finished in 72 s!

Finished in 72 s!

Running case A0.005730351121571223_B0.016474681583506928_DRS1.0372997336911433e-06_theta0.005431015294526998_fStar0.7547287416697764 on thread 12
Running case A0.014

Finished in 74 s!

Finished in 74 s!

Running case A0.009864858886162528_B0.018693784663674183_DRS2.2716890019397483e-06_theta0.009575034766361502_fStar0.6927055436270468 on thread 11
Running case A0.010640711002870392_B0.013483376537552928_DRS2.8358137110541e-06_theta0.005083663114266234_fStar0.6700783617843337 on thread 9
Finished in 73 s!

/home/shengduo/InverseProblems/GPRWorkingField
Finished in 73 s!

/home/shengduo/InverseProblems/GPRWorkingField
Finished in 74 s!

/home/shengduo/InverseProblems/GPRWorkingField
Finished in 72 s!

/home/shengduo/InverseProblems/GPRWorkingField
Finished in 73 s!

/home/shengduo/InverseProblems/GPRWorkingField
Finished in 73 s!

/home/shengduo/InverseProblems/GPRWorkingField
Finished in 73 s!

/home/shengduo/InverseProblems/GPRWorkingField
Finished in 73 s!

/home/shengduo/InverseProblems/GPRWorkingField
Finished in 73 s!

/home/shengduo/InverseProblems/GPRWorkingField
Finished in 73 s!

/home/shengduo/InverseProblems/GPRWorkingField
Finished in 73

In [255]:
# Test observations
# Test run a batch
input_set = [[0.016, 0.011, 0.1, 2], [0.016, 0.012, 0.2, 3]]
distanceAbove = 1e-3
nOfQueryPts = 10
fTerms = 16
obsFlag = 'frontsurf'
myBatch = RunABatch(input_set, work_path, fTerms, distanceAbove, nOfQueryPts, obsFlag)

/home/shengduo/InverseProblems/GPRWorkingField


In [257]:
testCase_idx = 0
u = myBatch.input_set[testCase_idx]
y = myBatch.Observations[testCase_idx]
np.linalg.norm(y)

9.169259194698046e-06

In [280]:
# Test open an h5 file
nOfQueryPts = 10

# Rotation matrix Q
alpha = 29 / 180 * np.pi
Q = np.array([[np.cos(alpha), np.sin(alpha)], [-np.sin(alpha), np.cos(alpha)]])

# VS start
VSstart = np.array([0.006354, 0.003522])

# Query points
distance_above = distanceAbove  # Distance above the interface

# Set x_up query points
x_up = 1e-3 * np.linspace(0., 45., nOfQueryPts)
QueryPts_up = np.stack([x_up, distance_above * np.ones(x_up.shape)], axis = 1)
QueryPts_dn = np.stack([x_up, -distance_above * np.ones(x_up.shape)], axis = 1)

# nOfNodes
nOfNodes = len(x_up)
        

# Open the file
# h5_file = "/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/2d/InverseExp/output/frontsurfFiles/A0.016_B0.011_fw0.1_Vw2-domain.h5"
h5_file = "/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/2d/InverseExp/output/frontsurfFiles/A0.016_B0.012_fw0.2_Vw3-domain.h5"

f = h5py.File(h5_file, 'r')

# Get time
times = np.array(f['time']).reshape([-1])
times = times - np.min(times)
nOfTSteps = times.shape[0]
times = times / np.max(times)

# Get coordinates
coords = np.array(f['geometry']['vertices']) - VSstart
Qcoords = coords @ Q.transpose()

# Store the slip rates
slip_rate_x = np.zeros([nOfTSteps, nOfNodes])

# Get Slip rates
velocity = np.array(f['vertex_fields']['velocity'])
Qvelocity = velocity @ Q.transpose()

for i in range(nOfTSteps):
    slip_rate_x[i, :] = - griddata(Qcoords, velocity[i, :, 0], QueryPts_up, method = 'cubic') \
                  + griddata(Qcoords, velocity[i, :, 0], QueryPts_dn, method = 'cubic')
slip_rate_x = slip_rate_x.transpose()

# Find the Fourier coefficients
FourierTerms = 32
T = np.max(times)

# Compute the Fourier terms
Ks = np.array(range(FourierTerms))
coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * times)
VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in slip_rate_x], 0)

# Compute the fourier coefficients
# print('time.shape: ', time.shape)
observation = np.trapz(VxcoskPiTt, x=times)

In [281]:
np.max(slip_rate_x)

0.26449251741463475

In [282]:
times

array([0.        , 0.01001001, 0.02102102, 0.03203203, 0.04304304,
       0.05405405, 0.06506507, 0.07607608, 0.08608609, 0.0960961 ,
       0.10610611, 0.11611612, 0.12612613, 0.13613614, 0.14614615,
       0.15615616, 0.16716717, 0.17817818, 0.18918919, 0.2002002 ,
       0.21121121, 0.22222222, 0.23323323, 0.24424424, 0.25525526,
       0.26626627, 0.27727728, 0.28828829, 0.2992993 , 0.30930931,
       0.31931932, 0.32932933, 0.33933934, 0.34934935, 0.35935936,
       0.36936937, 0.37937938, 0.38938939, 0.3993994 , 0.40940941,
       0.41941942, 0.42942943, 0.43943944, 0.44944945, 0.45945946,
       0.46946947, 0.47947948, 0.48948949, 0.4994995 , 0.50950951,
       0.51951952, 0.52952953, 0.53953954, 0.54954955, 0.55955956,
       0.56956957, 0.57957958, 0.58958959, 0.5995996 , 0.60960961,
       0.61961962, 0.62962963, 0.63963964, 0.64964965, 0.65965966,
       0.66966967, 0.67967968, 0.68968969, 0.6996997 , 0.70970971,
       0.71971972, 0.72972973, 0.73973974, 0.74974975, 0.75975

In [283]:
max(observation)

0.044417879449190616

In [284]:
np.linalg.norm(observation)

0.09185036106914132

In [138]:
[u >= myInv.u_low, u <= myInv.u_high]

[array([ True,  True,  True,  True]), array([ True,  True,  True,  True])]